In [9]:
%cd ..
%rm -rf no-time-to-backprop

/content


In [8]:
!ls
!nvidia-smi

constraints.txt			 requirements.txt
hpt.py				 results_for_rotational_vs_none
plotting_scripts		 run_all_benchmarks.sh
__pycache__			 run_all.sh
requirements-base.txt		 run_all_toy.sh
requirements.from_linux_env.txt  run_on_gpu.ipynb
requirements-linux-cuda.txt	 run_toy_task.py
requirements-macos-cpu.txt	 test_rec.py
requirements.mac.txt		 tgap
Thu Dec 18 22:40:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+=

In [10]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.80"

!git clone https://github.com/Francis2002/no-time-to-backprop.git
%cd no-time-to-backprop

Cloning into 'no-time-to-backprop'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 252 (delta 72), reused 75 (delta 39), pack-reused 138 (from 1)
Receiving objects: 100% (252/252), 91.75 MiB | 22.48 MiB/s, done.
Resolving deltas: 100% (141/141), done.
/content/no-time-to-backprop


In [11]:
!python -V
!ls

Python 3.12.12
constraints.txt			 requirements-macos-cpu.txt  run_on_gpu.ipynb
hpt.py				 requirements.mac.txt	     run_toy_task.py
plotting_scripts		 requirements.txt	     test_rec.py
requirements-base.txt		 run_all_benchmarks.sh	     tgap
requirements.from_linux_env.txt  run_all.sh
requirements-linux-cuda.txt	 run_all_toy.sh


In [12]:
%pip install -r requirements-linux-cuda.txt

In [13]:
import jax
import jaxlib
import sys

print(f"jax: {jax.__version__}")
print(f"jaxlib: {jaxlib.__version__}")
print(f"executable: {sys.executable}")
print(f"devices: {jax.devices()}")


jax: 0.8.1
jaxlib: 0.8.1
executable: /usr/bin/python3
devices: [CudaDevice(id=0)]


In [7]:
!python3 run_toy_task.py -m ONLINE -a ZUC --dataset mooc --task link_classification

[*] Results will be stored in /content/no-time-to-backprop/results_for_rotational_vs_none/mooc_ONLINE
[*] d_model set to 64
Running iteration: 0
[*] Output file /content/no-time-to-backprop/tgap/data/npzs/mooc_stream.npz already exists. Using existing file.
object address  : 0x7bd2cbad2f20
object refcount : 3
object type     : 0xa2a4e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C


In [ ]:
# Here we run experiments
!python3 run_toy_task.py -m ONLINE -a ZUC --num_layers 4 --num_hidden 64 --double_dmodel --mixing rotational_full --dataset mooc --batching_strategy rearranged --lr 0.001 --rec_learning_factor 0.5 --dropout 0.15 --weight_decay 0.1 --batch_size 50 --num_epochs 100 --num_gradient_accumulation_steps 16 --dedupe --pos_weight 50 --drop_hod_dow


[*] Results will be stored in /content/no-time-to-backprop/results_for_rotational_vs_none/mooc_ONLINE
[*] d_model set to 128

[*] ----------------------------- Iteration: 0 -----------------------------

[*] Npz file /content/no-time-to-backprop/tgap/data/npzs/mooc_stream.npz already exists with matching meta. Using existing file.
W1218 22:53:21.575212   24482 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1218 22:53:21.577272   24467 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.

[*] -------------------- Dataset Statistics --------------------
[*] Loaded data from /content/no-time-to-backprop/tgap/data/npzs/mooc_stream.npz
[*] Number of nodes: 7047
[*] Total number of edges: 8234
[*] Val_ratio: 0.2 | Test_ratio: 0.2
[*] Number of features: 6
[*] ------------------